In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import scipy.io
from sklearn.cross_validation import train_test_split

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/fixes.py:64: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  if 'order' in inspect.getargspec(np.copy)[0]:


In [3]:
raw_train_images = scipy.io.loadmat("dataset/train.mat")['train_images']
raw_train_labels = scipy.io.loadmat("dataset/train.mat")['train_labels']
shuff = np.random.choice(len(raw_train_labels), len(raw_train_labels), False)
raw_train_images = scipy.io.loadmat("dataset/train.mat")['train_images'].T.reshape(60000,784)[shuff]
raw_train_labels = scipy.io.loadmat("dataset/train.mat")['train_labels'].reshape(60000,)[shuff]
raw_train_labels = pd.get_dummies(pd.Series(raw_train_labels)).values

print(raw_train_images.shape, raw_train_labels.shape)
tX, tXt, ty, tyt = train_test_split(raw_train_images, raw_train_labels, test_size=0.33)
print(tX.shape, tXt.shape)
print(ty.shape, tyt.shape)

(60000, 784) (60000, 10)
(40200, 784) (19800, 784)
(40200, 10) (19800, 10)


In [4]:
print(scipy.io.loadmat("dataset/test.mat")['test_images'].T.shape)
kaggleTest = scipy.io.loadmat("dataset/test.mat")['test_images'].T.reshape(10000,784)
print(kaggleTest.shape)

(10000, 28, 28)
(10000, 784)


In [49]:
def add_bias(in_val):
    return np.vstack([in_val, np.ones(len(in_val[0]))])

def initialize_layer(input_dim, output_dim, seed):
    np.random.seed(seed)
    z = np.random.normal(0,0.01,output_dim)
    y = np.random.normal(0,0.01,output_dim)
    w = np.random.normal(0,0.01,(input_dim, output_dim))
    
    bz = np.append(np.random.normal(0,0.01,output_dim), 1)
    by = np.append(np.random.normal(0,0.01,output_dim), 1)
    bw = add_bias(np.random.normal(0,0.01,(input_dim, output_dim)))
    return z, bz, y, by, w, bw

In [50]:
def squared_loss_raw(y,y_2): return np.power((y - y_2), 2)
def squared_loss_deriv_raw(y,y_2): return (y_2 - y)
squared_loss = np.vectorize(squared_loss_raw)
squared_loss_deriv = np.vectorize(squared_loss_deriv_raw)

def calc_dEdz2(y_2, y, cost_deriv):
    return np.diag(
        np.multiply(y_2, np.subtract(1, y_2))
    ).dot(cost_deriv(y, y_2))

def calc_dEdz1(z_1, w_2, dEdz2):
    return np.diag(
        np.subtract(1, np.power(np.tanh(z_1), 2))
    ).dot(w_2).dot(dEdz2)

def predictNN(seq, X):
    temp = X
    for comm in seq:
        if "func" in str(type(comm)):
            temp = comm(temp)
        else:
            temp = temp.dot(comm)
    return temp.argmax(axis=1)

def scoreNN(seq, X, y):
    preds = predictNN(seq, X)
    y_vals = y.argmax(axis=1)
    return sum(preds != y_vals)/len(preds)

In [ ]:
params = {
    "hidden_layer_size": 200,
    "learning_rate":0.01,
    "epochs": 5,
    "epoch_length": 100,
    
    "cost_func": squared_loss,
    "cost_func_deriv": squared_loss_deriv
}

def trainNN(X, y, Xt, yt, params, seed=100):
    hl_size = params['hidden_layer_size']
    lr = params['learning_rate']
    epoch_length = params.get('epoch_length', len(X))
    feature_dim = len(tX[0])
    output_dim = len(y[0])
    
    cost = params['cost_func']
    cost_deriv = params['cost_func_deriv']

    z_1, bz_1, y_1, by_1, w_1, bw_1 = initialize_layer(feature_dim, hl_size, seed)
    z_2, bz_2, y_2, by_2, w_2, bw_2 = initialize_layer(hl_size, output_dim, seed)
    
    dEdw2 = np.zeros((hl_size, output_dim))
    dEdw1 = np.zeros((feature_dim, hl_size))
    print("-"*25, "Initialization") 
    print("Iterations per Epoch:", epoch_length)
    print("-"*25, "Training")

    for epoch in range(params['epochs']):
        print("Starting Epoch Number", epoch)
        for _ in range(epoch_length):
            training_point = int(np.random.choice(len(y), 1))
            y_0 = X[training_point]
            by_0 = np.append(y_0, 1)
            truth = y[training_point]

            # forward pass
            y_0.dot(w_1, out=z_1)
            np.tanh(z_1, out=y_1)
            y_1.dot(w_2, out=z_2)
            y_2 = 1 / (1 + np.exp(-1 * z_2)) # make it better?

            #cache
            dEdz2 = calc_dEdz2(y_2, truth, cost_deriv)
            dEdz1 = calc_dEdz1(z_1, w_2, dEdz2)

            #backprop
            np.outer(y_1, dEdz2.T, out=dEdw2) # l2 weights
            np.outer(y_0, dEdz1.T, out=dEdw1)# l1 weights

            #updates
            w_1 = np.subtract(w_1, np.multiply(lr, dEdw1))
            w_2 = np.subtract(w_2, np.multiply(lr, dEdw2))
            
        prediction_sequence = [w_1, np.tanh, w_2, np.vectorize(lambda x: 1 / 1 + np.exp(-1 * x))]
        print("Finished Epoch", epoch)
        train_score = scoreNN(prediction_sequence, X, y)
        print("Training Error", train_score)
        validation_score = scoreNN(prediction_sequence, Xt, yt)
        print("Validation Error", validation_score)
        print("-"*20)
        
    return [w_1, np.tanh, w_2, np.vectorize(lambda x: 1 / 1 + np.exp(-1 * x))]

seq = trainNN(tX, ty, tXt, tyt, params)

------------------------- Initialization
Iterations per Epoch: 100
------------------------- Training
Starting Epoch Number 0
Finished Epoch 0
Training Error 0.924527363184
Validation Error 0.923080808081
--------------------
Starting Epoch Number 1
Finished Epoch 1
Training Error 0.932985074627
Validation Error

In [ ]:
import datetime
kaggle_out = pd.DataFrame(predictNN(seq, kaggleTest)).reset_index()
kaggle_out.columns = ['Id', 'Category']
kaggle_out.Id = kaggle_out.Id + 1
t = str(datetime.datetime.now())
fname = "Kaggle prediction " + t + ".csv"
print(fname)

In [ ]:
kaggle_out.to_csv(fname,index=None)